Basic topic modelling for just the contexts and their topics


- right now running it without removing non-words!

Both global_tuning and evolutionary_tuning are set to True as a default, but can easily be changed. Perhaps you do not want the representations to be influenced by the global representation and merely see how they evolved over time:


to do if i care
- remove stopwords
- remove non-words somehow (not worth the computational effort)
- maybe make the fact/fiction also into periods

In [8]:
import pandas as pd
from pathlib import Path
from bertopic import BERTopic

rs_data = pd.concat([pd.read_excel("../data_for_viewing/extracted_raw/ff_royal_society_words_found_full_RSTB.xlsx"), pd.read_excel("../data_for_viewing/extracted_raw/ff_royal_society_words_found_full_RSTL.xlsx"), pd.read_excel("../data_for_viewing/extracted_raw/ff_royal_society_words_found_full_RSTA.xlsx")], ignore_index=True)
rs_data['Date'] = pd.to_datetime(rs_data['Date'], errors='coerce')
rs_data = rs_data.dropna(subset=['Date'])

# Fact
fact_data = rs_data[rs_data['Keyword'].str.contains('act', case=False, na=False)]
date_fact = fact_data['Date'].tolist()
contexts_fact = fact_data['Context'].to_list()

# Fiction
fiction_data = rs_data[rs_data['Keyword'].str.contains('ict', case=False, na=False)]
date_fict = fiction_data['Date'].tolist()
contexts_fict = fiction_data['Context'].to_list()

In [9]:
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from enchant.tokenize import get_tokenizer
import concurrent.futures

tknzr = get_tokenizer("en_US")
dictionary = Dict("en_US")

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def contains_digits(word):
    return any(char.isdigit() for char in word)

def remove_stop_words_and_non_words(text, stop_words):
    text = remove_punctuation(text)
    
    tokens = [word for word, _ in tknzr(text.lower())]
    
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    return ' '.join(filtered_tokens)

def process_texts(texts):
    cleaned_texts = []

    stop_words = set(stopwords.words('english')).union(ENGLISH_STOP_WORDS)
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_text = {executor.submit(remove_stop_words_and_non_words, text, stop_words): text for text in texts}
        
        for future in concurrent.futures.as_completed(future_to_text):
            cleaned_texts.append(future.result())
    
    return cleaned_texts

contexts_fact_clean = process_texts(contexts_fact)
contexts_fict_clean = process_texts(contexts_fict)


In [ ]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import concurrent.futures

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stop_words(text, stop_words):
    text = remove_punctuation(text)
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

def process_texts(texts):
    cleaned_texts = []
    stop_words = set(stopwords.words('english'))  # Using only NLTK stopwords
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_text = {executor.submit(remove_stop_words, text, stop_words): text for text in texts}
        
        for future in concurrent.futures.as_completed(future_to_text):
            cleaned_texts.append(future.result())
    
    return cleaned_texts

# Example usage with the variables contexts_fact and contexts_fict
contexts_fact_clean = process_texts(contexts_fact)
contexts_fict_clean = process_texts(contexts_fict)

In [10]:
# Fact (all together!, rstl, rstb, rsta)
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(contexts_fact_clean)
topics_over_time = topic_model.topics_over_time(contexts_fact_clean, date_fact, datetime_format="%Y-%m-%d", nr_bins=30)
fig1 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title= "Topics over time in Fact words found in Royal Society Transactions (RSTL, RSTB, RSTA)")
fig1.write_html('fact_topics_over_time_30.html')

fig1.show()

#topic_model.get_topic_info()

2024-09-18 10:42:56,293 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/1191 [00:00<?, ?it/s]

2024-09-18 10:44:11,432 - BERTopic - Embedding - Completed ✓
2024-09-18 10:44:11,433 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-09-18 10:44:28,436 - BERTopic - Dimensionality - Completed ✓
2024-09-18 10:44:28,437 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-09-18 10:44:34,027 - BERTopic - Cluster - Completed ✓
2024-09-18 10:44:34,038 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-09-18 10:44:44,572 - BERTopic - Representation - Completed ✓
30it [01:02,  2.07s/it]


In [11]:
# Fiction --meaningless due to low amount?
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(contexts_fict_clean)
topics_over_time = topic_model.topics_over_time(contexts_fict_clean, date_fict, datetime_format="%Y-%m-%d", nr_bins=30)
fig2 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title= "Topics over time in Fiction words found in Royal Society Transactions (RSTL, RSTB, RSTA)")
fig2.write_html('fiction_topics_over_time_30.html')

fig2.show()

2024-09-18 10:45:55,209 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/18 [00:00<?, ?it/s]

2024-09-18 10:45:58,074 - BERTopic - Embedding - Completed ✓
2024-09-18 10:45:58,075 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-09-18 10:46:01,289 - BERTopic - Dimensionality - Completed ✓
2024-09-18 10:46:01,293 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-09-18 10:46:01,311 - BERTopic - Cluster - Completed ✓
2024-09-18 10:46:01,316 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-09-18 10:46:01,494 - BERTopic - Representation - Completed ✓
25it [00:00, 32.47it/s]


THE TOPIC OF ALL FULL THE ROYAL SOCIETY ARTICLES?

In [12]:
#periods

import pandas as pd
from pathlib import Path
from bertopic import BERTopic

folders = {
    'rstb': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rstb'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rstb.csv'),
    'rstl': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rstl'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rstl.csv'),
    'rsta': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rsta'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rsta.csv')
}

def load_metadata(metadata_path):
    return pd.read_csv(metadata_path)

def load_texts_and_dates(folder_path, metadata_df):
    texts = []
    periods = []
    for txt_file in folder_path.glob('*.txt'):
        filename = txt_file.name
        file_date = metadata_df.loc[metadata_df['filename'] == filename.replace('.txt', '.pdf'), 'date'].values
        if file_date.size > 0:
            try:
                period = pd.Period(pd.to_datetime(file_date[0], format='%Y/%m/%d').strftime('%Y'), freq='Y')
                with open(txt_file, 'r', encoding='utf-8') as f:
                    texts.append(f.read())
                    periods.append(period)
                print(f"Processed file: {filename}")
            except (pd.errors.OutOfBoundsDatetime, ValueError):
                print(f"Invalid date for file: {filename}")
        else:
            print(f"Date not found for file: {filename}")
    return texts, periods

all_texts = []
all_periods = []
for key, (folder_path, metadata_path) in folders.items():
    metadata_df = load_metadata(metadata_path)
    texts, periods = load_texts_and_dates(folder_path, metadata_df)
    all_texts.extend(texts)
    all_periods.extend(periods)
    print(f"Processed {len(texts)} files from {folder_path}")

all_texts_clean = [process_texts(text) for text in all_texts]

period_strings = [period.strftime('%Y-%m-%d') for period in all_periods]

topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(all_texts_clean)

topics_over_time = topic_model.topics_over_time(all_texts_clean, period_strings, datetime_format="%Y-%m-%d", nr_bins=30)
fig4 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title="Topics over time in Fact and Fiction texts (RSTL, RSTB, RSTA)")
fig4.write_html('full_rs_topics_over_time_periods_30.html')

fig4.show()

Processed file: rstb1990_355_1397_549.txt
Processed file: rstb_1887_0001.txt
Processed file: rstb_1887_0007.txt
Processed file: rstb_1887_0013.txt
Processed file: rstb_1887_0008.txt
Processed file: rstb_1887_0006.txt
Processed file: rstb_1887_0012.txt
Processed file: rstb_1887_0003.txt
Processed file: rstb_1887_0015.txt
Processed file: rstb_1887_0009.txt
Processed file: rstb_1887_0005.txt
Processed file: rstb_1887_0011.txt
Processed file: rstb_1887_0010.txt
Processed file: rstb_1887_0002.txt
Processed file: rstb_1887_0016.txt
Processed file: rstb_1888_0002.txt
Processed file: rstb_1887_0018.txt
Processed file: rstb_1888_0005.txt
Processed file: rstb_1887_0004.txt
Processed file: rstb_1888_0008.txt
Processed file: rstb_1888_0007.txt
Processed file: rstb_1888_0006.txt
Processed file: rstb_1888_0004.txt
Processed file: rstb_1887_0017.txt
Processed file: rstb_1888_0014.txt
Processed file: rstb_1888_0001.txt
Processed file: rstb_1888_0013.txt
Processed file: rstb_1888_0011.txt
Processed fil

In [12]:
#periods

import pandas as pd
from pathlib import Path
from bertopic import BERTopic

folders = {
    'rstb': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rstb'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rstb.csv'),
    'rstl': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rstl'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rstl.csv'),
}

def load_metadata(metadata_path):
    return pd.read_csv(metadata_path)

def load_texts_and_dates(folder_path, metadata_df):
    texts = []
    periods = []
    for txt_file in folder_path.glob('*.txt'):
        filename = txt_file.name
        file_date = metadata_df.loc[metadata_df['filename'] == filename.replace('.txt', '.pdf'), 'date'].values
        if file_date.size > 0:
            try:
                period = pd.Period(pd.to_datetime(file_date[0], format='%Y/%m/%d').strftime('%Y'), freq='Y')
                with open(txt_file, 'r', encoding='utf-8') as f:
                    texts.append(f.read())
                    periods.append(period)
                print(f"Processed file: {filename}")
            except (pd.errors.OutOfBoundsDatetime, ValueError):
                print(f"Invalid date for file: {filename}")
        else:
            print(f"Date not found for file: {filename}")
    return texts, periods

all_texts = []
all_periods = []
for key, (folder_path, metadata_path) in folders.items():
    metadata_df = load_metadata(metadata_path)
    texts, periods = load_texts_and_dates(folder_path, metadata_df)
    all_texts.extend(texts)
    all_periods.extend(periods)
    print(f"Processed {len(texts)} files from {folder_path}")

all_texts_clean = [process_texts(text) for text in all_texts]

period_strings = [period.strftime('%Y-%m-%d') for period in all_periods]

topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(all_texts_clean)

topics_over_time = topic_model.topics_over_time(all_texts_clean, period_strings, datetime_format="%Y-%m-%d", nr_bins=30)
fig5 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title="Topics over time in Fact and Fiction texts (RSTL, RSTB)")
fig5.write_html('rstbl_topics_over_time_periods_30.html')

fig5.show()

Processed file: rstb1990_355_1397_549.txt
Processed file: rstb_1887_0001.txt
Processed file: rstb_1887_0007.txt
Processed file: rstb_1887_0013.txt
Processed file: rstb_1887_0008.txt
Processed file: rstb_1887_0006.txt
Processed file: rstb_1887_0012.txt
Processed file: rstb_1887_0003.txt
Processed file: rstb_1887_0015.txt
Processed file: rstb_1887_0009.txt
Processed file: rstb_1887_0005.txt
Processed file: rstb_1887_0011.txt
Processed file: rstb_1887_0010.txt
Processed file: rstb_1887_0002.txt
Processed file: rstb_1887_0016.txt
Processed file: rstb_1888_0002.txt
Processed file: rstb_1887_0018.txt
Processed file: rstb_1888_0005.txt
Processed file: rstb_1887_0004.txt
Processed file: rstb_1888_0008.txt
Processed file: rstb_1888_0007.txt
Processed file: rstb_1888_0006.txt
Processed file: rstb_1888_0004.txt
Processed file: rstb_1887_0017.txt
Processed file: rstb_1888_0014.txt
Processed file: rstb_1888_0001.txt
Processed file: rstb_1888_0013.txt
Processed file: rstb_1888_0011.txt
Processed fil

2024-09-17 19:11:21,521 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/405 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#periods

import pandas as pd
from pathlib import Path
from bertopic import BERTopic

folders = {
    'rsta': (Path('D:/Fact_fiction_corpus/texts/royal society/txt_rsta'), 'D:/Fact_fiction_corpus/texts/royal society/royalsociety_metadata_rsta.csv')
}

def load_metadata(metadata_path):
    return pd.read_csv(metadata_path)

def load_texts_and_dates(folder_path, metadata_df):
    texts = []
    periods = []
    for txt_file in folder_path.glob('*.txt'):
        filename = txt_file.name
        file_date = metadata_df.loc[metadata_df['filename'] == filename.replace('.txt', '.pdf'), 'date'].values
        if file_date.size > 0:
            try:
                period = pd.Period(pd.to_datetime(file_date[0], format='%Y/%m/%d').strftime('%Y'), freq='Y')
                with open(txt_file, 'r', encoding='utf-8') as f:
                    texts.append(f.read())
                    periods.append(period)
                print(f"Processed file: {filename}")
            except (pd.errors.OutOfBoundsDatetime, ValueError):
                print(f"Invalid date for file: {filename}")
        else:
            print(f"Date not found for file: {filename}")
    return texts, periods

all_texts = []
all_periods = []
for key, (folder_path, metadata_path) in folders.items():
    metadata_df = load_metadata(metadata_path)
    texts, periods = load_texts_and_dates(folder_path, metadata_df)
    all_texts.extend(texts)
    all_periods.extend(periods)
    print(f"Processed {len(texts)} files from {folder_path}")

all_texts_clean = [process_texts(text) for text in all_texts]

period_strings = [period.strftime('%Y-%m-%d') for period in all_periods]

topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(all_texts_clean)

topics_over_time = topic_model.topics_over_time(all_texts_clean, period_strings, datetime_format="%Y-%m-%d", nr_bins=30)
fig6 = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20, title="Topics over time in Fact and Fiction texts (RSTA)")
fig6.write_html('rsta_topics_over_time_periods_30.html')

fig6.show()